# Explicit_schemes

This validation file aims at illustrating the importance of the stratagey used for explicit schemes (see comments in Schema_Euler_explicite.cpp, method faire_un_pas_de_temps_eqn_base)

```
  /*
   * The strategy done in the explicit schemes is to prescribe CL at time n+1
   *  at the begining of the time step, turning the wheel (avancer/reculer) to use
   *  it in derivee_en_temps_inco (so that valeurs() points to futur() only for BC).
   *
   * It influences only two cases
   *     - Time dependent BCs (champ_fonc_txyz ou ICoCo) or function (champ_fonc_fonction) => Here it only shifts the
   *        applied BC by one time step, but does not change the order of the scheme
   *
   *     - Coupled cases (paroi contact) => Here it is mandatory to have the equality of the fluxes at the coupled boundary
   *
   *     See the out files of docond/docond_vef
   */
   ```
   
For that, we select a coupled solid/fluid problem and we test all explicit schemes, both in VDF and VEF. Both, the full explicit and the diffusion-implicit options are considered. The equality of the fluxes at the contact boundary is used to justify the strategy chosen. Additional temperature profiles in both problems are also used for comparisons.


# Attention

The schemes are explicit and the stability of the scheme is required. Attention if you miss-understood the facsec keyword. For simplicity, refer to it as a CFL condition. Thats why we use

| Scheme | facsec (CFL) |
|--------|--------------|
| Euler  | 1            |
| RRK2   | 0.5          |
| RK2    | 0.5          |
| RK2    | 0.4          |
| RK4    | 0.3          |
| AB2    | 0.25         |
| AB3    | 0.1          |


# Remember! A higher-order time scheme provides better accuracy, but it does not allow for larger time steps!

In [ ]:
from trustutils import run

run.introduction("Elie Saikali & Adrien Bruneton")
run.TRUST_parameters()

In [ ]:
import pandas as pd
import os, sys
from string import Template

run.reset()
run.initBuildDirectory()

dis = [("VDF", "", "-"), ("VEF", "Trianguler_H dom_solide Trianguler_H dom_fluide", "--")]
diffusion = [("Explicit", "0"), ("Diffusion_implicit", "1")]
schema = [
            ("Euler", "Scheme_euler_explicit", "1") 
          , ("Rational_RK2", "Runge_Kutta_Rationnel_ordre_2", "0.5")
          , ("RK2", "Runge_Kutta_ordre_2_classique", "0.5")
          , ("RK3", "Runge_Kutta_ordre_3_classique", "0.4")
          , ("RK4", "Runge_Kutta_ordre_4_classique", "0.3")
          , ("RK4_38", "Runge_Kutta_ordre_4_classique_3_8", "0.3")
          , ("AB2", "schema_adams_bashforth_order_2", "0.25")
          , ("AB3", "schema_adams_bashforth_order_3", "0.1")
        # , ("RK2_Low", "Runge_Kutta_ordre_2", "1.5")
        # , ("RK3_Low", "Runge_Kutta_ordre_3", "1.5")
        # , ("RK4_Low", "Runge_Kutta_ordre_4", "1.5")
          ]

N_VDF = ["8 22", "16 8", "16 16"]
N_VEF = ["4 11", "8 4", "8 8"]

for d, p, _ in dis:
    for sc, sch, fsc in schema:
        for ex, di in diffusion:
            if d == "VDF":
                run.addCaseFromTemplate("jdd.data", f"{d}/{sc}/{ex}", {"trian" : p, "dis" : d, "diff" : di, "sch" : sch, "fsc" : fsc, "N1": "7 21", "N2" : "15 7", "N3" : "15 15"})
            else:
                run.addCaseFromTemplate("jdd.data", f"{d}/{sc}/{ex}", {"trian" : p, "dis" : d, "diff" : di, "sch" : sch, "fsc" : fsc, "N1": "4 11", "N2" : "8 4", "N3" : "8 8"})
        
run.printCases()
run.runCases()
run.tablePerf()

# Convergence

We plot the residu evolution. All simulations converge!

In [ ]:
from trustutils import plot
import numpy as np

a = plot.Graph(label_size=16, title_size=24, legend_size=14, nY=2)
for i, (ex,_) in enumerate(diffusion):
    a.addPlot(i,f"Convergence: {ex}")
    for d, _, m in dis:
        for sc, _, _ in schema:
            a.addResidu(f"{run.BUILD_DIRECTORY}/{d}/{sc}/{ex}/jdd.dt_ev", label=f"{d}/{sc}", marker=m, linewidth=(2 if d == "VDF" else 1))
    a.scale(yscale="log")
    a.legend(loc='upper center', bbox_to_anchor=(0.5, 1.), ncol=3, fancybox=True, shadow=True)

# Time step evolution (in seconds)

In [ ]:
a = plot.Graph(label_size=16, title_size=24, legend_size=14, nY=2)
for i, (ex,_) in enumerate(diffusion):
    a.addPlot(i,f"Time steps: {ex}")
    for d, _, m in dis:
        for sc, _, _ in schema:
            dt_ev = np.loadtxt(f"{run.BUILD_DIRECTORY}/{d}/{sc}/{ex}/jdd.dt_ev")
            a.add(dt_ev[:, 0], dt_ev[:, 1], label=f"{d}/{sc}", marker=m)
            a.legend(loc='upper center', bbox_to_anchor=(0.5, 0.9), ncol=3, fancybox=True, shadow=True)
            a.label("Time [s]","Time step [s]")

# Flux evolution on contact boundary: both fluid/solid problems

In all cases the equality of the fluxes is seen for each time scheme.

Much better agreement with full explicit schemes. In VEF, attention to the result analysis when using a diffusion-implicit scheme.

In [ ]:
a = plot.Graph(label_size=16, title_size=24, legend_size=14, nY=2)
for i, (ex,_) in enumerate(diffusion):
    a.addPlot(i,f"Flux pb1 vs pb2: {ex}")
    for d, _, m in dis:
        for sc, _, _ in schema:
            fpb2 = np.loadtxt(f"{run.BUILD_DIRECTORY}/{d}/{sc}/{ex}/jdd_pb2_Diffusion_chaleur.out")
            a.add(fpb2[:, 0], fpb2[:, 4], label=f"{d}/{sc} - pb 2", marker=m, linewidth=(2 if d == "VDF" else 1) )
    
            fpb1 = np.loadtxt(f"{run.BUILD_DIRECTORY}/{d}/{sc}/{ex}/jdd_pb1_Diffusion_chaleur.out")
            a.add(fpb1[:, 0], -1*fpb1[:, 3], label=f"{d}/{sc} pb 1", marker=m, linewidth=(2 if d == "VDF" else 1))
            a.label("Time [s]","Heat transfer rate [W]")
            a.legend(loc='upper center', bbox_to_anchor=(0.5, 1.5), ncol=3, fancybox=True, shadow=True)



# Vertical profiles - Solid problem

In [ ]:
a = plot.Graph(label_size=16, title_size=24, legend_size=14, nY=2)
for i, (ex,_) in enumerate(diffusion):
    a.addPlot(i,f"Temperature evolution pb solide : {ex}")
    for d, _, m in dis:
        for sc, _, _ in schema:
            a.addSegment(f"{run.BUILD_DIRECTORY}/{d}/{sc}/{ex}/jdd_SOL1.son",label=f"{d}/{sc} - pb 1", marker=m,compo=0, linewidth=(2 if d == "VDF" else 1))
            a.legend(loc='upper center', bbox_to_anchor=(0.5, 1.3), ncol=3, fancybox=True, shadow=True)

# Horizontal profiles - Solid problem

In [ ]:
a = plot.Graph(label_size=16, title_size=24, legend_size=14, nY=2)
for i, (ex,_) in enumerate(diffusion):
    a.addPlot(i,f"Temperature evolution pb solide : {ex}")
    for d, _, m in dis:
        for sc, _, _ in schema:
            a.addSegment(f"{run.BUILD_DIRECTORY}/{d}/{sc}/{ex}/jdd_SOL2.son",label=f"{d}/{sc} - pb 1", marker=m,compo=0, linewidth=(2 if d == "VDF" else 1))
            a.legend(loc='upper center', bbox_to_anchor=(0.5, 1.3), ncol=3, fancybox=True, shadow=True)

# Vertical profiles - Fluid problem

In [ ]:
a = plot.Graph(label_size=16, title_size=24, legend_size=14, nY=2)
for i, (ex,_) in enumerate(diffusion):
    a.addPlot(i,f"Temperature evolution pb solide : {ex}")
    for d, _, m in dis:
        for sc, _, _ in schema:
            a.addSegment(f"{run.BUILD_DIRECTORY}/{d}/{sc}/{ex}/jdd_FL2.son",label=f"{d}/{sc} - pb 1", marker=m,compo=0, linewidth=(2 if d == "VDF" else 1))
            a.legend(loc='upper center', bbox_to_anchor=(0.5, 1.3), ncol=3, fancybox=True, shadow=True)

# Horizontal profiles - Fluid problem

In [ ]:
a = plot.Graph(label_size=16, title_size=24, legend_size=14, nY=2)
for i, (ex,_) in enumerate(diffusion):
    a.addPlot(i,f"Temperature evolution pb solide : {ex}")
    for d, _, m in dis:
        for sc, _, _ in schema:
            a.addSegment(f"{run.BUILD_DIRECTORY}/{d}/{sc}/{ex}/jdd_FL1.son",label=f"{d}/{sc} - pb 1", marker=m,compo=0, linewidth=(2 if d == "VDF" else 1))
            a.legend(loc='upper center', bbox_to_anchor=(0.5, 1.3), ncol=3, fancybox=True, shadow=True)